# Content
This notebook contains a script to derive soil properties according to:
- Gazetas, G. (1991). Formulas and charts for impedances of surface and embedded foundations. Journal of Geotechnical Engineering, 117(9), 1363–1381. https://doi.org/10.1061/(asce)0733-9410(1991)117:9(1363`
  
Input parameters are from `CIEM5220_SRE Project_Part 2_2025.pdf`

In [ ]:
# Soil Properties (indipendent of Student Number)
nu = 0.33      #[-] Poisson Ratio
c_u = 50*10e3  #[Pa] Undrained Shear Strength


# Foundation dimensions (rectangular slab shallow foundation)
side_x = side_y = 4.94 #[m] As a first implementation the slab is as large as the base of the tower
# depth  not necessary
 

In [ ]:
L = side_x/2      #[m] Half-length between center of the circumscribed rectangle and side X-direction (major direction)
B = side_y/2      #[m] Half-width between center of the circumscribed rectangle and side Y-direction (minor direction)
I_bx = L*B**3/12  #[m^4] Moment of inertia about x-axis
I_by = L**3*B/12  #[m^4] Moment of inertia about y-axis
G = c_u

# Dynamic Stiffness
K_y = (2*G*L*/(2-nu))()


